<a href="https://colab.research.google.com/github/abojha/AbOjha/blob/main/DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.9 MB/s eta 0:00:00


In [14]:
from datasets import load_dataset, Dataset
import pandas as pd
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer
import subprocess

In [15]:
def load_and_preprocess_data(csv_path="/content/drive/MyDrive/dl/data.csv"):
    # Check if preprocessed CSV exists
    if os.path.exists(csv_path):
        print(f"Loading preprocessed data from {csv_path}")
        return pd.read_csv(csv_path)

    # Load and preprocess fresh data
    print("Preprocessing dataset...")
    dataset = load_dataset("sahil2801/CodeAlpaca-20k", split="train")

    # Convert to pandas DataFrame and format Q&A pairs
    df = dataset.to_pandas()
    df["question"] = "Question: " + df["instruction"] + " " + df["input"].apply(
        lambda x: x if str(x).strip() != "" else ""
    )
    df["answer"] = "Answer: " + df["output"]
    df = df[["question", "answer"]].dropna().drop_duplicates()

    # Save to CSV
    df.to_csv(csv_path, index=False)
    print(f"Saved preprocessed data to {csv_path}")
    return df

In [16]:
df = load_and_preprocess_data()

Preprocessing dataset...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/147 [00:00<?, ?B/s]

code_alpaca_20k.json:   0%|          | 0.00/8.06M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/20022 [00:00<?, ? examples/s]

Saved preprocessed data to /content/drive/MyDrive/dl/data.csv


In [28]:
df = pd.read_csv("/content/drive/MyDrive/dl/data.csv")

In [30]:
print(df.head(5))   # First 5 rows
print(df.tail(5))   # Last 5 rows


                                            question  \
0  Question: Create an array of length 5 which co...   
1  Question: Formulate an equation to calculate t...   
2  Question: Write a replace method for a string ...   
3  Question: Create an array of length 15 contain...   
4  Question: Write a function to find the number ...   

                                              answer  
0                     Answer: arr = [2, 4, 6, 8, 10]  
1  Answer: Height of triangle = opposite side len...  
2  Answer: def replace(self, replace_with):\n    ...  
3  Answer: arr = [3, 6, 9, 12, 15, 18, 21, 24, 27...  
4  Answer: def find_num_distinct_states(matrix):\...  
                                                question  \
20017  Question: Identify and display all the data ty...   
20018  Question: Create a basic Python script to acce...   
20019  Question: Create a basic C# program to print o...   
20020  Question: Find and display all the employees w...   
20021  Question: Create a Python 

In [32]:
df = df.reset_index(drop=True)  # Reset to sequential indices
df = df.iloc[1000:2000]  # Now slice the next 1000 rows
print(df)  # Should print data now


                                               question  \
1000  Question: Reverse the string given in the inpu...   
1001  Question: Create a class that creates a stack ...   
1002  Question: Implement the code in the input to t...   
1003  Question: Find the minimum of two given number...   
1004  Question: Sort the given array using insertion...   
...                                                 ...   
1995  Question: Given two string parameters,  s1 and...   
1996  Question: Create a CSS rule that makes all h2'...   
1997  Question: Create an algorithm to generate all ...   
1998  Question: Suggest a possible solution to the f...   
1999  Question: Identify the names of different clas...   

                                                 answer  
1000  Answer: s = "Hello world" \ns = s[::-1] \nprin...  
1001  Answer: class Stack: \n\tdef __init__(self): \...  
1002  Answer: def do_something(data):\n    # Your co...  
1003  Answer: def min_two(a, b):\n  if a < b:\n    r...  
1

In [33]:
df.head()

,question,answer
1000,Question: Reverse the string given in the inpu...,"Answer: s = ""Hello world"" \ns = s[::-1] \nprin..."
1001,Question: Create a class that creates a stack ...,Answer: class Stack: \n\tdef __init__(self): \...
1002,Question: Implement the code in the input to t...,Answer: def do_something(data):\n # Your co...
1003,Question: Find the minimum of two given number...,"Answer: def min_two(a, b):\n if a < b:\n r..."
1004,Question: Sort the given array using insertion...,"Answer: def insertion_sort(arr):\n """"""\n ..."


In [39]:
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def initialize_model():
    model_path = "/content/drive/MyDrive/dl/my_model"  # Check correct folder name

    if os.path.exists(model_path):
        print("✅ Loading fine-tuned model from:", model_path)
        tokenizer = AutoTokenizer.from_pretrained(model_path)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    else:
        print("⚠️ Fine-tuned model not found, loading base model instead.")
        model_name = "Salesforce/codet5-base"
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    return tokenizer, model

# Example usage:
tokenizer, model = initialize_model()


✅ Loading fine-tuned model from: /content/drive/MyDrive/dl/my_model


In [41]:
# 3. Data Tokenization
# --------------------
def tokenize_data(df, tokenizer):
    def tokenize_function(examples):
        inputs = tokenizer(
            examples["question"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
        outputs = tokenizer(
            examples["answer"],
            max_length=128,
            truncation=True,
            padding="max_length"
        )
        return {
            "input_ids": inputs.input_ids,
            "attention_mask": inputs.attention_mask,
            "labels": outputs.input_ids
        }

    dataset = Dataset.from_pandas(df)
    return dataset.map(tokenize_function, batched=True)

  # Tokenize data
tokenized_dataset = tokenize_data(df, tokenizer)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [43]:
# 4. Model Training
# --------------------
def train_model(tokenized_dataset, model):
    # Split dataset
    split_dataset = tokenized_dataset.train_test_split(test_size=0.1)
    train_dataset, val_dataset = split_dataset["train"], split_dataset["test"]

    # Training configuration
    training_args = TrainingArguments(
        output_dir="/content/drive/MyDrive/dl/my_model",
        num_train_epochs=5,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        learning_rate=3e-4,
        fp16=True,
        logging_steps=100,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        report_to="none"
    )

    # Initialize trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )

    # Start training
    trainer.train()
    return model


model.save_pretrained("/content/drive/MyDrive/dl/my_model")
tokenizer.save_pretrained("/content/drive/MyDrive/dl/my_model")

In [44]:
# Train model (comment out after first training)
model = train_model(tokenized_dataset, model)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.265500,0.184236


Epoch,Training Loss,Validation Loss
1,0.265500,0.184236
2,0.190700,0.184415
3,0.128700,0.176415
4,0.075300,0.175169
5,0.047500,0.172142


In [46]:
# --------------------
# 5. Inference & Chat
# --------------------
class CodeAssistant:
    def __init__(self, model, tokenizer):
        self.model = model
        self.tokenizer = tokenizer
        self.model.eval()

    def generate_answer(self, question):
        input_text = f"Question: {question}"
        input_ids = self.tokenizer.encode(input_text, return_tensors="pt").to(self.model.device)
        outputs = self.model.generate(input_ids, max_length=128)
        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def execute_code(self, code):
        try:
            with open("temp.py", "w") as f:
                f.write(code)
            result = subprocess.run(["python", "temp.py"], capture_output=True, text=True)
            return f"Output:\n{result.stdout}\nErrors:\n{result.stderr}"
        except Exception as e:
            return f"Error: {str(e)}"

    def respond(self, question):
        answer = self.generate_answer(question)
        if "```python" in answer:
            code = answer.split("```python")[1].split("```")[0].strip()
            execution_result = self.execute_code(code)
            return f"{answer}\n\n{execution_result}"
        return answer

# --------------------
# 6. Gradio Interface
# --------------------
def launch_interface(assistant):
    return gr.Interface(
        fn=assistant.respond,
        inputs=gr.Textbox(lines=2, placeholder="Ask a coding question..."),
        outputs="text",
        title="Code Assistant",
        examples=[
            ["Write a Python function to reverse a string"],
            ["How to calculate factorial in JavaScript?"],
            ["Explain recursion with an example"]
        ]
    )

In [47]:
# Initialize assistant
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
model_path = "/content/drive/MyDrive/dl/my_model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

assistant = CodeAssistant(model, tokenizer)

# Launch interface
interface = launch_interface(assistant)
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2a792b4c7a94e941ae.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
